In [13]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup

In [14]:
def siksin(place):
    base_url = 'https://www.siksinhot.com/search'
    url = f'{base_url}?keywords={quote(place)}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    lis = soup.select('.localFood_list > li')
    line = []
    for li in lis:
        img = li.find('img')['src']
        href = li.select_one('figure').find('a')['href']
        title = li.select_one('.textBox > h2').get_text()
        score = li.select_one('.textBox > .score').get_text()
        atags = li.select('.cate > a')
        location = atags[0].get_text()
        menu = atags[1].get_text()
        line.append({'img': img, 'title': title, 'score': score, 'location': location, 'menu': menu, 'href': href})
    
    return line

In [15]:
def lyrics(artist, title):
    base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query='
    url = f'{base_url}{quote(artist+title)}+%EA%B0%80%EC%82%AC'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    lyr = soup.select_one('.lyrics_txt._lyrics_txt').get_text().strip()

    return lyr

In [20]:
import re
import joblib
new_cvect = joblib.load('model/imdb_cvect_2.pkl')
new_lrc = joblib.load('model/imdb_lrc2.pkl')

In [22]:
text = lyrics('에스파', 'life too short')
text = re.sub('[^A-Za-z]', ' ', text)
review_cv = new_cvect.transform([text])
'긍정' if new_lrc.predict(review_cv)[0] == 1 else '부정'

'긍정'

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
a = cvect.fit_transform([text]).toarray()[0]
print(a)
print(cvect.vocabulary_)

[1 2 1 1 6 1 1 4 9 2 1 6 1 1 1 1 3 1 1 6 3 6 4 2 3 1 1 3 1 2 1 2 2 1 3 1 1
 2 2 3 3 2 3 1 7 1 1 2 2 1 6 1 1 6 1 1 1 2 1 1 1 2 1 3 1 1 1 3 1 1 1]
{'discussiontalking': 6, 'nothingyou': 33, 'say': 44, 'faceyou': 10, 'think': 56, 'words': 70, 'gospelbut': 15, 'troublei': 58, 'ain': 0, 'got': 16, 'time': 57, 'wasteyou': 64, 'need': 31, 'lifecause': 23, 'life': 22, 'shortyou': 48, 'bored': 1, 'mindyou': 29, 'really': 38, 'nonsensesomewhere': 32, 'elsecause': 9, 've': 61, 'realizei': 37, 'doing': 7, 'regardlessand': 40, 'don': 8, 'carewhat': 4, 'itand': 21, 'matterif': 27, 'like': 24, 'noti': 34, 'having': 19, 'fun': 11, 'sowhy': 50, 'stopdoing': 53, 'regardlessno': 42, 'imma': 20, 'waythat': 67, 'wanti': 63, 'stopsome': 55, 'people': 35, 'meanall': 28, 'phone': 36, 'screenwhen': 45, 'tryna': 59, 'live': 25, 'liveswhy': 26, 'gotta': 17, 'viciousbe': 62, 'business': 2, 'stead': 52, 'getting': 12, 'mineyou': 30, 'regardlessdoing': 41, 'regardlesswhy': 43, 'stopoh': 54, 'won': 69, 'turn': 60, '

In [36]:
for i in range(len(a)):
    i == cvect.vocabulary_.values()

71